<a href="https://colab.research.google.com/github/lucienwalewski/CSE303/blob/master/interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# load tracks
import pickle
with open('drive/MyDrive/Friends/tracks-features/Friends.pk', 'rb') as f:
  tracks = pickle.load(f)

In [ ]:
! mkdir data
! mkdir data/frames
! tar xzf drive/MyDrive/Friends/frames/episode01.tar.gz -C data/frames

In [ ]:
from google.colab import files

In [ ]:
import os

! mkdir data/frames/cropped_videos

for track_no in tracks['episode01']['body']:

  DATA_PATH = 'data/frames/episode01'
  OUT_PATH = 'data/frames/track' + str(track_no).zfill(3)
  ! mkdir $OUT_PATH
    
  count = 0
  for frame_bbx in tracks['episode01']['body'][track_no]:
    frame_no = frame_bbx[0]
    x1, y1, x2, y2 = frame_bbx[1], frame_bbx[2], frame_bbx[3], frame_bbx[4]

    width = (x2 - x1)
    height = (y2 - y1)

    img_fname = str(int(frame_no)).zfill(6) + ".jpg"
    in_img = os.path.join(DATA_PATH, img_fname)
    out_fno = str(int(count)).zfill(4) + ".jpg"
    out_img = os.path.join(OUT_PATH, f"cropped_{out_fno}")
    crop_str = f'"crop={width}:{height}:{x1}:{y1}"'

    ! ffmpeg -i $in_img -vf $crop_str $out_img

    count += 1

  input_path = f'data/frames/track{str(track_no).zfill(3)}/cropped_%4d.jpg'
  video_path = f'data/frames/cropped_videos/track{str(track_no).zfill(3)}.mp4'
  ! ffmpeg -f image2 -i $input_path $video_path

  files.download(video_path)



In [ ]:
import os

! mkdir data/frames/cropped_videos_resized

for track_no in tracks['episode01']['body']:

  DATA_PATH = 'data/frames/episode01'
  OUT_PATH = 'data/frames/track' + str(track_no).zfill(3) + '_resized'
  ! mkdir $OUT_PATH
    
  count = 0
  for frame_bbx in tracks['episode01']['body'][track_no]:
    frame_no = frame_bbx[0]

    x1, y1, x2, y2 = frame_bbx[1], frame_bbx[2], frame_bbx[3], frame_bbx[4]

    width = (x2 - x1)
    height = (y2 - y1)
    width_n = width * 1.2
    height_n = height * 1.2
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2

    x1_n = x_center - (width_n / 2)
    x2_n = x_center + (width_n / 2)

    if x1_n < 0 or x2_n > 1280:
      width_n = width
      x1_n = x1

    y1_n = y_center - (height_n / 2) 
    y2_n = y_center + (height_n / 2)

    if y1_n < 0 or y2_n > 720:
      height_n = height
      y1_n = y1

    img_fname = str(int(frame_no)).zfill(6) + ".jpg"
    in_img = os.path.join(DATA_PATH, img_fname)
    out_fno = str(int(count)).zfill(4) + "_resized.jpg"
    out_img = os.path.join(OUT_PATH, f"cropped_{out_fno}")

    crop_str = f'"crop={width_n}:{height_n}:{x1_n}:{y1_n}"'
    print(crop_str)

    assert x1_n + width_n <= 1280
    assert y1_n + height_n <= 720

    ! ffmpeg -i $in_img -vf $crop_str $out_img

    count += 1

  input_path = f'data/frames/track{str(track_no).zfill(3)}_resized/cropped_%4d_resized.jpg'
  video_path = f'data/frames/cropped_videos_resized/track{str(track_no).zfill(3)}_resized.mp4'
  ! ffmpeg -f image2 -i $input_path $video_path

  files.download(video_path)
